In [0]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
!cp gdrive/My\ Drive/ADBI-Submission/amazonreviews.zip .
!unzip amazonreviews.zip

Archive:  amazonreviews.zip
replace test.ft.txt.bz2? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: test.ft.txt.bz2         
replace train.ft.txt.bz2? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: train.ft.txt.bz2        


In [0]:
import numpy as np

# To store data
import pandas as pd

# To open zipped files
import bz2

# To use regular expressions
import re

#To load and save data
import pickle

In [0]:
# Open file and read lines
train_file = bz2.BZ2File('train.ft.txt.bz2')
test_file = bz2.BZ2File('test.ft.txt.bz2')


In [0]:
# Takes arnd 4-5 minutes to read 4 million reviews
train_reviews = train_file.readlines()
test_reviews = test_file.readlines()


In [0]:
# Converting Raw strings to strings that can be parsed and is suitable for ML
train_reviews = [x.decode('utf-8') for x in train_reviews]
test_reviews = [x.decode('utf-8') for x in test_reviews]

In [0]:
with open('train_reviews.pkl', 'wb') as f:
  pickle.dump(train_reviews, f)

In [0]:
with open('test_reviews.pkl', 'wb') as f:
  pickle.dump(test_reviews, f)

In [0]:
# Persist the train files to google drive [VERY IMPORTANT]
!cp train_reviews.pkl gdrive/My\ Drive/ADBI-Submission/Preprocessing/Data/
# Persist the test file to google drive [VERY IMPORTANT]
!cp test_reviews.pkl gdrive/My\ Drive/ADBI-Submission/Preprocessing/Data/

In [0]:
# Retrievng train_reviews to store the labels too using pickle
!cp gdrive/My\ Drive/ADBI-Submission/Preprocessing/Data/train_reviews.pkl .

#Loading data
with open('train_reviews.pkl', 'rb') as f:
  train_reviews = pickle.load(f)

In [0]:
x = train_reviews[0]
train_label = [0 if x.split(' ')[0] == '__label__1' else 1]
train_sentence = [x.split(' ', 1)[1][:-1].lower()] # Adding [:-1] removes new line character at the end

print(train_sentence,train_label)

['stuning even for the non-gamer: this sound track was beautiful! it paints the senery in your mind so well i would recomend it even to people who hate vid. game music! i have played the game chrono cross but out of all of the games i have ever played it has the best music! it backs away from crude keyboarding and takes a fresher step with grate guitars and soulful orchestras. it would impress anyone who cares to listen! ^_^'] [1]


In [0]:
# Spliting reviews and labels. 

# Labels - Negetive = 0, Positive = 1
train_review_labels = [0 if x.split(' ')[0] == '__label__1' else 1 for x in train_reviews]
train_review_sentences = [x.split(' ', 1)[1][:-1].lower() for x in train_reviews]

In [0]:
with open('train_review_labels.pkl', 'wb') as f:
  pickle.dump(train_review_labels, f)

In [0]:
# Storing it in drive
!cp train_review_labels.pkl gdrive/My\ Drive/ADBI-Submission/Preprocessing/Data/

In [0]:
len(train_reviews), len(train_review_labels)

# 3600000, 3600000

(3600000, 3600000)

In [0]:
# Takes around 2 minutes - Preprocessing of data

# Not removing non-word character beacuse can't will change to can t and removing single characters with space will change meaning from can't to can
# Will do it using regex tokenizer during tokenization process
for i in range(len(train_review_sentences)):
  train_review_sentences[i] = re.sub('\d',' ',train_review_sentences[i]) # Replacing digits by space
  train_review_sentences[i] = re.sub(r'\s+[a-z][\s$]', ' ',train_review_sentences[i]) # Removing single characters and spaces alongside
  train_review_sentences[i] = re.sub(r'\s+', ' ',train_review_sentences[i]) # Replacing more than one space with a single space

In [0]:
# Pre-processing for URL's : Replacing the  url by <url> key word
for i in range(len(train_review_sentences)):
    if 'www.' in train_review_sentences[i] or 'http:' in train_review_sentences[i] or 'https:' in train_review_sentences[i] or '.com' in train_review_sentences[i]:
        train_review_sentences[i] = re.sub(r"([^ ]+(?<=\.[a-z]{3}))", "<url>", train_review_sentences[i])
        
        
train_review_sentences[101:105]

In [0]:
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [0]:
#Removal of stopwords - 1 min

cachedStopWords = set(stopwords.words("english"))
for i in range(len(train_review_sentences)):
  word_list = train_review_sentences[i].split(' ')
  filtered_words = [word for word in word_list if word not in cachedStopWords]
  train_review_sentences[i] = ' '.join(filtered_words)

In [0]:
# Removal of punctuations - Can do this by using tokenizer as i will do below
import string 
def remove_punc(s):
    revw = str.maketrans({key: None for key in string.punctuation})
    return s.translate(revw)

In [0]:
# Storing preprocessed reviews

with open('preprocessed_train_review_sentences.pkl', 'wb') as f:
  pickle.dump(train_review_sentences, f)


In [0]:
# Persist the file to google drive [VERY IMPORTANT]
!cp preprocessed_train_review_sentences.pkl gdrive/My\ Drive/ADBI-Submission/Preprocessing/Data/

In [0]:
# Retrive the processed train reviews and doing some data analysis

!cp gdrive/My\ Drive/ADBI-Submission/Preprocessing/Data/preprocessed_train_review_sentences.pkl .
!cp gdrive/My\ Drive/ADBI-Submission/Preprocessing/Data/train_review_labels.pkl .

In [0]:
with open('preprocessed_train_review_sentences.pkl', 'rb') as f:
    train_review_sentences = pickle.load(f)

In [0]:
with open('train_review_labels.pkl', 'rb') as f:
    train_review_labels = pickle.load(f)

In [0]:
# Analysing the data
train_reviews_df = pd.DataFrame(
    {'review': train_review_sentences,
     'label': train_review_labels,
    })


In [0]:
# Analyzing data via dataframe operations
train_reviews_df.head()

,review,label
0,stuning even non-gamer: sound track beautiful!...,1
1,best soundtrack ever anything.: i'm reading lo...,1
2,"amazing!: soundtrack favorite music time, hand...",1
3,excellent soundtrack: truly like soundtrack en...,1
4,"remember, pull jaw floor hearing it: played ga...",1


In [0]:
train_reviews_df.tail()

,review,label
3599995,it!!: high chair looks great first comes box h...,0
3599996,"looks nice, low functionality: used highchair ...",0
3599997,"compact, hard clean: small house, really wante...",0
3599998,saying?: sure book supposed be. really rehash ...,0
3599999,makes blood run red-white-and-blue: agree ever...,1


In [0]:
train_reviews_df.describe() # Ensures we have only values between 0,1

,label
count,3600000.0
mean,0.5
std,0.5
min,0.0
25%,0.0
50%,0.5
75%,1.0
max,1.0


In [0]:
train_reviews_df['label'].value_counts()
# Ensures we have only 2 labels positive and negetive
# Shows that we have equal number of positive and negetive reviews. So no skewness in data. We have equal number of samples which is a good thing

1    1800000
0    1800000
Name: label, dtype: int64

 Tokenize the data to make it ML model ready
 Create word vocab to make it ready for next step of Glove Word Embeddings 

In [0]:
# Use Tokenizer to remove punctuations and non-word characters and tokenize the text
import os
os.environ['KERAS_BACKEND']='theano' # Or TenserFlow
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

Using Theano backend.


In [0]:
MAX_SEQUENCE_LENGTH = 100
MAX_NB_WORDS = 8000 # This specifies how many top tokens in each review to be stored. Wrongly interpreted as total number of words(token) together in whole dataset
EMBEDDING_DIM = 100

In [0]:
# Takes 5 minutes to run on entire training dataset
tokenizer = Tokenizer(num_words=MAX_NB_WORDS)
tokenizer.fit_on_texts(train_review_sentences)
train_sequences = tokenizer.texts_to_sequences(train_review_sentences)

word_index = tokenizer.word_index

print('Number of Unique Tokens',len(word_index)) # Total 996497 unique words 
# 996497

In [0]:
with open('Word_Index.pkl', 'wb') as f:
  pickle.dump(word_index, f)
  
# Persist the file to google drive [VERY IMPORTANT]
!cp Word_Index.pkl gdrive/My\ Drive/ADBI-Submission/Preprocessing/Data/

In [0]:
# To overcome above issue, we pad the sequences so that all have same length
train_sequences_padded = pad_sequences(train_sequences, maxlen=MAX_SEQUENCE_LENGTH)

In [0]:
with open('Tokenized_Padded_Train_Sequence.pkl', 'wb') as f:
  pickle.dump(train_sequences_padded, f)

# Persist the file to google drive [VERY IMPORTANT]
!cp Tokenized_Padded_Train_Sequence.pkl gdrive/My\ Drive/ADBI-Submission/Preprocessing/Data/

Working with Test data to preprocess and convert to padded sequences

In [0]:
# Load Test Reviews
!cp gdrive/My\ Drive/ADBI-Submission/Preprocessing/Data/test_reviews.pkl .

In [0]:
with open('test_reviews.pkl', 'rb') as f:
  test_reviews = pickle.load(f)

In [0]:
# Spliting reviews and labels and preprocessing the text reviews to remove url's, non-word characters, converting to lower case etc.

# Labels - Negetive = 0, Positive = 1
test_review_labels = [0 if x.split(' ')[0] == '__label__1' else 1 for x in test_reviews]
test_review_sentences = [x.split(' ', 1)[1][:-1].lower() for x in test_reviews]

In [0]:
for i in range(len(test_review_sentences)):
  test_review_sentences[i] = re.sub('\d',' ',test_review_sentences[i]) # Replacing digits by space
  test_review_sentences[i] = re.sub(r'\s+[a-z][\s$]', ' ',test_review_sentences[i]) # Removing single characters and spaces alongside
  test_review_sentences[i] = re.sub(r'\s+', ' ',test_review_sentences[i]) # Replacing more than one space with a single space

In [0]:
# Pre-processing for URL's : Replacing the  url by <url> key word
for i in range(len(test_review_sentences)):
    if 'www.' in test_review_sentences[i] or 'http:' in test_review_sentences[i] or 'https:' in test_review_sentences[i] or '.com' in test_review_sentences[i]:
        test_review_sentences[i] = re.sub(r"([^ ]+(?<=\.[a-z]{3}))", "<url>", test_review_sentences[i])

In [0]:
# Removal of stop words
cachedStopWords = set(stopwords.words("english"))
for i in range(len(test_review_sentences)):
  word_list = test_review_sentences[i].split(' ')
  filtered_words = [word for word in word_list if word not in cachedStopWords]
  test_review_sentences[i] = ' '.join(filtered_words)

In [0]:
# Dumping to drive
with open('test_review_labels.pkl', 'wb') as f:
  pickle.dump(test_review_labels, f)

In [0]:
# Storing it in drive
!cp test_review_labels.pkl gdrive/My\ Drive/ADBI-Submission/Preprocessing/Data/

In [0]:
with open('preprocessed_test_review_sentences.pkl', 'wb') as f:
  pickle.dump(test_review_sentences, f)

!cp preprocessed_test_review_sentences.pkl gdrive/My\ Drive/ADBI-Submission/Preprocessing/Data/

In [0]:
# Creating Tokenized, padded sequence for test_reviews
test_sequences = tokenizer.texts_to_sequences(test_review_sentences)
test_sequences_padded = pad_sequences(test_sequences, maxlen=MAX_SEQUENCE_LENGTH)

In [0]:
len(test_sequences_padded[0]),len(test_sequences_padded[1]),len(test_sequences_padded) # All lengths are 100 as defined and length = input test size = 0.4M

In [0]:
# Saving Test Padded Sequences to Drive
with open('Tokenized_Padded_Test_Sequence.pkl', 'wb') as f:
  pickle.dump(test_sequences_padded, f)

!cp Tokenized_Padded_Test_Sequence.pkl gdrive/My\ Drive/ADBI-Submission/Preprocessing/Data/

Loading Glove model and creating embedding matrix

In [0]:
!cp gdrive/My\ Drive/ADBI-Submission/glove100d.zip .
!unzip glove100d.zip

In [0]:
# Creating dictionary of words and corresponding vectors
embeddings_index = {}
for i, line in enumerate(open('glove.6B.100d.txt')):
  values = line.split() # 0 th index will be the word and rest will the embedding vector (size 100 as we have used Glove.6B.100D embedding file) 
  embeddings_index[values[0]] = np.asarray(values[1:], dtype='float32')

In [0]:
# create token(words in word index)-embedding mapping
embedding_matrix = np.zeros((len(word_index) + 1, 100)) # 100 since embedding_dimesion is 100, +1 because index 0 is reserved in word_index
for word, i in word_index.items():
  embedding_vector = embeddings_index.get(word)
  if embedding_vector is not None:
    embedding_matrix[i] = embedding_vector
# We can initialize random vector and assign for words which are not present in embeddings.Other option is keep trainable=true in embedding layer of the NN model.
# We choose 2nd option

In [0]:
# storing the word-embedding mapping matrix
with open('WordEmbeddingMappingMatrix.pkl', 'wb') as f:
  pickle.dump(embedding_matrix, f)

In [0]:
# Storing to drive
!cp WordEmbeddingMappingMatrix.pkl gdrive/My\ Drive/ADBI-Submission/Preprocessing/Data/

In [0]:
# To load it from drive

!cp gdrive/My\ Drive/ADBI-Submission/Preprocessing/Data/WordEmbeddingMappingMatrix.pkl .
with open('WordEmbeddingMappingMatrix.pkl', 'rb') as f:
    embedding_matrix = pickle.load(f)

In [0]:
!cp gdrive/My\ Drive/ADBI-Submission/Preprocessing/Data/Word_Index.pkl .
with open('Word_Index.pkl', 'rb') as f:
    word_index = pickle.load(f)

In [0]:
nonzero_elements = np.count_nonzero(np.count_nonzero(embedding_matrix, axis=1))
nonzero_elements / len(word_index) # 0.18 coverage of words in embeddings
# So keep trainable = true in the NN model

0.1806508198218359

# Preproceeing done. We can now create and validate our models by loading required train, test sequences from google drive directly